In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [12]:
# Load the diabetes dataset.
ds = datasets.load_diabetes(as_frame=True)
print(f"The dataset contains the following information: {ds.keys()}")

# Check total number of rows and columns in the dataset.
print(f"Total number of samples inside data is '{ds.data.shape[0]}' and there are '{ds.data.shape[1]}' attributes to inference the '{ds.target.name}' column.", end="\n\n")

# Obtain a brief look of the dataset.
print(f"Visualize the first few samples: \n. {ds.frame.head()}")

# (Optional) See dataset description.
# print(f"Dataset contextual information:\n {df.DESCR}")

The dataset contains the following information: dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])
Total number of samples inside data is '442' and there are '10' attributes to inference the 'target' column.

Visualize the first few samples: 
.         age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  target  
0 -0.002592  0.019907 -0.017646   151.0  
1 -0.039493 -0.068332 -0.092204    75.0  
2 -0.002592  0.002861 -0.025930   141.0  
3  0.034309  0.022688 -0.009362   206.0  
4 -0.002592 -0.031988 -0.0466

In [ ]:
# Efectuate the partition of the dataset into training and testing data.
x_train, y_train, x_test, y_test = train_test_split(ds.data, ds.target, test_size=0.2, random_state=42)
print(f"Training data shape: {x_train.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Testing data shape: {x_test.shape}")
print(f"Testing target shape: {y_test.shape}")

Training data shape: (353, 10)
Training target shape: (89, 10)
Testing data shape: (353,)
Testing target shape: (89,)


In [29]:
# Define the model using the Tensorflow library.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Regression output.
])

# Compile the model.
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

# Train the model.
model.fit(x_train, x_test, epochs=50, validation_split=0.2)

# Evaluate the model.
loss, mae = model.evaluate(y_train, y_test)
print(f"Test Loss: {loss} \n Test MAE: {mae}")

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 31718.0078 - mae: 159.8566 - val_loss: 22466.2773 - val_mae: 134.0236
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 33401.4258 - mae: 163.6038 - val_loss: 22435.6309 - val_mae: 133.9102
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 31673.0352 - mae: 158.7820 - val_loss: 22392.7598 - val_mae: 133.7521
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 33003.8320 - mae: 163.8864 - val_loss: 22330.8652 - val_mae: 133.5247
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 30749.4922 - mae: 157.4286 - val_loss: 22242.2012 - val_mae: 133.1999
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 31775.3984 - mae: 159.4357 - val_loss: 22118.0293 - val_mae: 132.7457
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 31363.7754 - mae: 158.6015 - val_loss: 21949.9023 - val_mae: 132.1301
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 32635.1270 - mae: 162.5044 - val_loss: 21727.6172 - va